Part 2 concerns adding measurement error into the simulation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [132]:
iterations = 10000
n = 100
secret_beta = 2

$$
y = \beta x + \epsilon
$$

We only have data on $\tilde{x} = x + u$, where $u \sim N(0, \sigma_u^2)$

The OLS estimator for $\beta$ is

$$
\hat{\beta} = \frac{cov(\tilde{x}, y)}{var(\tilde{x})} = \frac{cov(x + u, \beta x + \epsilon)}{var(\tilde{x})}
$$

$$
plim \hat{\beta} = \frac{\sigma^2_x}{\sigma^2_x + \sigma^2_u} \beta
$$

Thus, the OLS estimator $\hat{\beta}$ is biased. The bias is $\hat{\beta} - \beta = -\frac{\sigma^2_u}{\sigma^2_x + \sigma^2_u} \beta$

Also, $\sigma^2_x + \sigma^2_u = var(\tilde{x})$

If $\sigma^2_u$ is known, we can obtain an unbiased estimator $\tilde{\beta} = \hat{\beta} \cdot \frac{var(\tilde{x})}{var(\tilde{x}) - \sigma^2_u}$

Here we try different $\sigma^2_u$ and $\sigma^2_\epsilon$ and calculate the bias and MSE before and after adjusting for biasness.

Estimating the residual variance:

$\hat{\epsilon} = y - \hat{\beta}\tilde{x} = y - \hat{\beta}(x+u)$

$\epsilon = y - \beta x$

$\hat{\epsilon} = y - \hat{\beta}\tilde{x} = y - \hat{\beta}(x+u) + (\epsilon - (y - \beta x)) = \epsilon + (\beta - \hat{\beta}) - \hat{\beta} u$

The residual contains two additional sources of variation compared to the true error.

- $(\beta - \hat{\beta})$: $\hat{\beta}$ is biased towards zero, under measurement error the term $\hat{\beta} - \beta$ does not vanish asymptotically
- Additional variance due to measurement error in the regressor

$\text{plim } \hat{\sigma^2_\epsilon} = \sigma^2_\epsilon + (1-\lambda)^2 \beta^2 \sigma_x^2 + \lambda^2 \beta^2 \sigma^2_u$

Thus, the estimated $\sigma^2_\epsilon$ is biased upwards.

$\lambda = \frac{\sigma^2_x}{\sigma^2_x + \sigma^2_u}$

In [140]:
print("Normally distributed epsilon")

for var_u in [0, 1.5, 2, 2.5]:
    for var_eps in [1.5, 2, 2.5]:
        bias_beta = np.array([])
        sqerr_beta = np.array([])
        bias_beta_adj = np.array([])
        sqerr_beta_adj = np.array([])
        
        bias_sigma = np.array([])
        bias_sigma_adj = np.array([])
        
        for _ in range(iterations):
            x = np.linspace(-10,10,n)
            x_obs = x + np.random.normal(scale=np.sqrt(var_u), size=n)
            y = x*secret_beta + np.random.normal(scale=np.sqrt(var_eps), size=n)

            beta_est = np.cov(x_obs, y)[0][1]/np.var(x_obs, ddof=1)
            beta_est_adj = np.cov(x_obs, y)[0][1]/np.var(x_obs, ddof=1) * np.var(x_obs, ddof=1)/(np.var(x_obs, ddof=1) - var_u)
            
            sigma_est = np.sum((y - beta_est * x_obs) ** 2) / (n-1)
            sigma_est_adj = np.sum((y - beta_est_adj * x_obs) ** 2) / (n-1)
            
            bias_beta = np.append(bias_beta, (beta_est-secret_beta))
            sqerr_beta = np.append(sqerr_beta, (beta_est-secret_beta)**2)
            bias_beta_adj = np.append(bias_beta_adj, (beta_est_adj-secret_beta))
            sqerr_beta_adj = np.append(sqerr_beta_adj, (beta_est_adj-secret_beta)**2)
            
            bias_sigma = np.append(bias_sigma, sigma_est-var_eps)
            bias_sigma_adj = np.append(bias_sigma_adj, sigma_est_adj-var_eps)

        print("var_u=%.2f, var_eps=%.2f:  Unadjusted: Bias = %f, MSE = %f; Sigma bias = %f"%(var_u, var_eps, bias_beta.mean(), sqerr_beta.mean(), bias_sigma.mean()))
    
        print("var_u=%.2f, var_eps=%.2f:  Adjusted:   Bias = %f, MSE = %f; Sigma bias = %f"%(var_u, var_eps, bias_beta_adj.mean(), sqerr_beta_adj.mean(), bias_sigma_adj.mean()))

Normally distributed epsilon
var_u=0.00, var_eps=1.50:  Unadjusted: Bias = -0.000338, MSE = 0.000439; Sigma bias = -0.001362
var_u=0.00, var_eps=1.50:  Adjusted:   Bias = -0.000338, MSE = 0.000439; Sigma bias = -0.001362
var_u=0.00, var_eps=2.00:  Unadjusted: Bias = 0.000294, MSE = 0.000593; Sigma bias = 0.006379
var_u=0.00, var_eps=2.00:  Adjusted:   Bias = 0.000294, MSE = 0.000593; Sigma bias = 0.006379
var_u=0.00, var_eps=2.50:  Unadjusted: Bias = -0.000154, MSE = 0.000727; Sigma bias = 0.003370
var_u=0.00, var_eps=2.50:  Adjusted:   Bias = -0.000154, MSE = 0.000727; Sigma bias = 0.003370
var_u=1.50, var_eps=1.50:  Unadjusted: Bias = -0.082730, MSE = 0.008739; Sigma bias = 5.756851
var_u=1.50, var_eps=1.50:  Adjusted:   Bias = 0.001188, MSE = 0.002357; Sigma bias = 6.014166
var_u=1.50, var_eps=2.00:  Unadjusted: Bias = -0.082322, MSE = 0.008851; Sigma bias = 5.759395
var_u=1.50, var_eps=2.00:  Adjusted:   Bias = 0.001637, MSE = 0.002566; Sigma bias = 6.016850
var_u=1.50, var_eps=2.5

$var = \frac{df}{df-2}$

$df = \frac{2 \cdot var}{var - 1}$

In [141]:
print("T distributed epsilon")
for var_u in [0, 1.5, 2, 2.5]:
    for var_eps in [1.5, 2, 2.5]:
        bias_beta = np.array([])
        sqerr_beta = np.array([])
        bias_beta_adj = np.array([])
        sqerr_beta_adj = np.array([])
        
        bias_sigma = np.array([])
        bias_sigma_adj = np.array([])
        
        for _ in range(iterations):
            x = np.linspace(-10,10,n)
            if var_u > 0:
                x_obs = x + np.random.standard_t(df=2*var_u/(var_u-1), size=n)
            else:
                x_obs = x
            y = x*secret_beta + np.random.standard_t(df=2*var_eps/(var_eps-1), size=n)

            beta_est = np.cov(x_obs, y)[0][1]/np.var(x_obs, ddof=1)
            beta_est_adj = np.cov(x_obs, y)[0][1]/np.var(x_obs, ddof=1) * np.var(x_obs, ddof=1)/(np.var(x_obs, ddof=1) - var_u)
            
            sigma_est = np.sum((y - beta_est * x_obs) ** 2) / (n-1)
            sigma_est_adj = np.sum((y - beta_est_adj * x_obs) ** 2) / (n-1)
            
            bias_beta = np.append(bias_beta, (beta_est-secret_beta))
            sqerr_beta = np.append(sqerr_beta, (beta_est-secret_beta)**2)
            bias_beta_adj = np.append(bias_beta_adj, (beta_est_adj-secret_beta))
            sqerr_beta_adj = np.append(sqerr_beta_adj, (beta_est_adj-secret_beta)**2)
            
            bias_sigma = np.append(bias_sigma, sigma_est-var_eps)
            bias_sigma_adj = np.append(bias_sigma_adj, sigma_est_adj-var_eps)

        print("var_u=%.2f, var_eps=%.2f:  Unadjusted: Bias = %f, MSE = %f; Sigma bias = %f"%(var_u, var_eps, bias_beta.mean(), sqerr_beta.mean(), bias_sigma.mean()))
    
        print("var_u=%.2f, var_eps=%.2f:  Adjusted:   Bias = %f, MSE = %f; Sigma bias = %f"%(var_u, var_eps, bias_beta_adj.mean(), sqerr_beta_adj.mean(), bias_sigma_adj.mean()))

T distributed epsilon
var_u=0.00, var_eps=1.50:  Unadjusted: Bias = 0.000120, MSE = 0.000443; Sigma bias = 0.002170
var_u=0.00, var_eps=1.50:  Adjusted:   Bias = 0.000120, MSE = 0.000443; Sigma bias = 0.002170
var_u=0.00, var_eps=2.00:  Unadjusted: Bias = -0.000229, MSE = 0.000589; Sigma bias = -0.004330
var_u=0.00, var_eps=2.00:  Adjusted:   Bias = -0.000229, MSE = 0.000589; Sigma bias = -0.004330
var_u=0.00, var_eps=2.50:  Unadjusted: Bias = 0.000337, MSE = 0.000742; Sigma bias = 0.005373
var_u=0.00, var_eps=2.50:  Adjusted:   Bias = 0.000337, MSE = 0.000742; Sigma bias = 0.005373
var_u=1.50, var_eps=1.50:  Unadjusted: Bias = -0.081403, MSE = 0.008722; Sigma bias = 5.738366
var_u=1.50, var_eps=1.50:  Adjusted:   Bias = 0.002650, MSE = 0.002605; Sigma bias = 5.996283
var_u=1.50, var_eps=2.00:  Unadjusted: Bias = -0.081448, MSE = 0.008844; Sigma bias = 5.740207
var_u=1.50, var_eps=2.00:  Adjusted:   Bias = 0.002591, MSE = 0.002726; Sigma bias = 5.998189
var_u=1.50, var_eps=2.50:  Unadj

Conclusion 1: MSE for T-distributed measurement errors and eps error is greater than Normal-distributed errors for all variance combination experimented.

Conclusion 2: Estimation bias for $\sigma^2_\epsilon$ for T-distributed errors is smaller than that for Normal-distributed errors.

Maybe because T-distribution has greater Kurtosis than normal distribution?

"Kurtosis is a statistical measure that defines how heavily the tails of a distribution differ from the tails of a normal distribution. In other words, kurtosis identifies whether the tails of a given distribution contain extreme values."